##Instalamos Librerías

In [ ]:
!pip install pymupdf nltk

## Añadimos el documento escaneado

In [ ]:
from google.colab import files

uploaded = files.upload()



Saving el principito-español-free-editorial.pdf to el principito-español-free-editorial.pdf


In [ ]:
import os
file_name = next(iter(uploaded))


## Se divide  el documento por páginas para no saturar el api de ChatGpt y se tranforma a formato imagen .png

In [ ]:
import fitz  # Importa PyMuPDF

try:
    # Abre el documento original
    doc = fitz.open(file_name)

    # Itera sobre cada página en el documento
    for page_num in range(doc.page_count):
        # Obtener la página actual
        page = doc.load_page(page_num)

        # Convertir la página en una imagen de mapa de bits (pixmap)
        pix = page.get_pixmap()

        # Guardar la imagen en formato PNG
        filename = f"Page_{page_num + 1:02}.png"
        pix.save(filename)



    # Cierra el documento original
    doc.close()
except Exception as e:
    print(f"An error occurred: {e}")

## Se instala las últimas actualizaciones de la librería de Openai de manera  silenciosa.

In [ ]:
!pip install --upgrade openai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.8/367.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00


 ### Se emplea el  API de OpenAI para aprovechar su  capacidad de procesamiento de imágenes para extraer y procesar texto de imágenes, y luego se segmenta el texto utilizando NLTK

In [ ]:
import os
import base64
import openai
import re
from natsort import natsorted
from nltk.tokenize import PunktSentenceTokenizer

# Configuración de la API de OpenAI
api_key = '$OPENAI_API_KEY' #cambia por tu clave.
client = openai.OpenAI(api_key=api_key)
MODEL = "gpt-4o"

segmentador=PunktSentenceTokenizer()

def encode_image(image_path):
    """ Codifica una imagen a base64. """
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

def preprocess_text(text):
    # Elimina guiones al final de la línea y unir con la palabra siguiente
    text = re.sub(r'-\s*\n\s*', '', text)
    # Elimina líneas completamente vacías
    text = re.sub(r'^\s*$\n', '', text, flags=re.MULTILINE)
    return text
# Archivo para guardar los resultados
with open("extracted_texts.txt", "w") as output_file:
    # Itera sobre cada archivo en el directorio ordenado naturalmente
    for filename in natsorted(os.listdir(os.getcwd())):
        if filename.endswith(".png"):
            # Codifica la imagen subida
            base64_image = encode_image(filename)

            # Crea la solicitud de chat para extraer texto
            try:
                response = client.chat.completions.create(
                    model=MODEL,
                    messages=[
                        {"role": "system", "content": "Eres un útil asistente que extrae texto de imágenes"},
                        {"role": "user", "content": [
                            {"type": "text", "text": '''Extraer el texto de la imagen escrito en un idioma que nunca has visto antes. Por favor, asegúrese de extraer el texto exactamente como aparece en la imagen, carácter por carácter, sin intentar predecir, modificar o corregir las palabras.Cada letra debe reconocerse tal como está escrita, aunque parezca incorrecta o desconocida. No confundas la letra "u" con la "v". Proporciona únicamente el texto extraído, respetando estrictamente estos criterios. '''},
                            {"type": "image_url", "image_url": {
                                "url": f"data:image/png;base64,{base64_image}"}
                            }
                        ]}
                    ],
                    temperature=0.5

                )

                # Verifica y guarda la respuesta
                if response.choices:
                    extracted_text = response.choices[0].message.content
                    processed_text = preprocess_text(extracted_text)
                    # Aplicar el segmentador al texto extraído
                    segmentos = segmentador.tokenize(processed_text)
                    formatted_text = "\n".join(segmentos)
                    output_file.write(f"{formatted_text}\n\n")
                else:
                    output_file.write(f"No response received for {filename}.\n\n")
                print(filename)
            except Exception as e:
                output_file.write(f"An error occurred while processing {filename}: {str(e)}\n\n")
        print(filename)




.config
Page_01.png
Page_01.png
Page_02.png
Page_02.png
Page_03.png
Page_03.png
Page_04.png
Page_04.png
Page_05.png
Page_05.png
Page_06.png
Page_06.png
Page_07.png
Page_07.png
Page_08.png
Page_08.png
Page_09.png
Page_09.png
Page_10.png
Page_10.png
Page_11.png
Page_11.png
Page_12.png
Page_12.png
Page_13.png
Page_13.png
Page_14.png
Page_14.png
Page_15.png
Page_15.png
Page_16.png
Page_16.png
Page_17.png
Page_17.png
Page_18.png
Page_18.png
Page_19.png
Page_19.png
Page_20.png
Page_20.png
Page_21.png
Page_21.png
Page_22.png
Page_22.png
Page_23.png
Page_23.png
Page_24.png
Page_24.png
Page_25.png
Page_25.png
Page_26.png
Page_26.png
Page_27.png
Page_27.png
Page_28.png
Page_28.png
Page_29.png
Page_29.png
Page_30.png
Page_30.png
Page_31.png
Page_31.png
Page_32.png
Page_32.png
Page_33.png
Page_33.png
Page_34.png
Page_34.png
Page_35.png
Page_35.png
Page_36.png
Page_36.png
Page_37.png
Page_37.png
Page_38.png
Page_38.png
Page_39.png
Page_39.png
Page_40.png
Page_40.png
Page_41.png
Page_41.png
Page_42.

In [ ]:
files.download('extracted_texts.txt')

FileNotFoundError: Cannot find file: extracted_texts.txt